<a href="https://colab.research.google.com/github/UmarBalak/MoodMapper/blob/main/Senti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#V1

In [ ]:
import re
import warnings
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
warnings.filterwarnings("ignore", category=FutureWarning)
import keras

In [ ]:
df = pd.read_csv('senti_large.csv', encoding = "ISO-8859-1")

In [ ]:
df = df[['text', 'sentiment']]

In [ ]:
df.head(20)

,text,sentiment
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0
5,Need a hug,0
6,@LOLTrish hey long time no see! Yes.. Rains a...,0
7,@Tatiana_K nope they didn't have it,0
8,@twittera que me muera ?,0
9,spring break in plain city... it's snowing,0


In [ ]:
df['text'] = df['text'].map(lambda x: x.lstrip('@VirginAmerica@UnitedAir@Southwestairline@DeltaAir@USAirways@American').rstrip('@'))

In [ ]:
df.head()

,text,sentiment
0,upset that he can't update his Facebook by te...,0
1,Kenichan I dived many times for the ball. Mana...,0
2,whole body feels itchy and like its on fire,0
3,"no, it's not behaving at all. i'm mad. why am...",0
4,Kwesidei not the whole crew,0


In [ ]:
dfs = df.sample(frac=1).reset_index(drop=True)
dfs.head()

,text,sentiment
0,Morning It's a beautiful day!,4
1,joined twitter follow @bekclinkaberry,4
2,up and awake but has a busy day ahead of her...,0
3,Cruel Intentions is over (i want to watch Sta...,0
4,folk Shame... does keep the idea floating in y...,4


In [ ]:
mapping = {0: 'negative', 4: 'positive'}

dfs['sentiment'] = dfs['sentiment'].replace(mapping)

In [ ]:
dfs.head(20)

,text,sentiment
0,Morning It's a beautiful day!,positive
1,joined twitter follow @bekclinkaberry,positive
2,up and awake but has a busy day ahead of her...,negative
3,Cruel Intentions is over (i want to watch Sta...,negative
4,folk Shame... does keep the idea floating in y...,positive
5,Nakonek Hey! I will be up moving into my apt o...,positive
6,i dont feel well... lol,negative
7,fire alarms seriously hurt your ears. A LOT. ...,negative
8,Hangin with some friends jacuuuuzi time,positive
9,Lenny_o i know! i would be so pissed if i had ...,negative


In [ ]:
df['text'] = df['text'].apply(lambda x: x.lower())

In [ ]:
df['text'] = df['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [ ]:
max_fatures = 4000
tokenizer = Tokenizer(num_words = max_fatures, split=' ')
tokenizer.fit_on_texts(dfs['text'].values)
X = tokenizer.texts_to_sequences(dfs['text'].values)
X = pad_sequences(X)
Y = dfs['sentiment']
L = Y.values
X
L

array(['positive', 'positive', 'negative', ..., 'positive', 'negative',
       'positive'], dtype=object)

In [ ]:
dfs.shape

(1599999, 2)

In [ ]:
k = []
for i in range(1599999):
    if L[i]=="negative":
                        k.append(0)
    elif L[i]=="positive":
                        k.append(1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, k,
                                                    test_size=0.3,
                                                    shuffle = True,
                                                    stratify = k,
                                                    random_state = 1 )

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
type(Y_train)

list

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
embed_dim = 128
lstm_out = 196
model = keras.models.Sequential()
model.add(keras.layers.Embedding(max_fatures, 128, input_length=X_train.shape[1]))
model.add(keras.layers.SpatialDropout1D(0.3))
model.add(keras.layers.LSTM(128, dropout = 0.3, recurrent_dropout = 0.3 ))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(100, activation = tf.nn.relu))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(2, activation = tf.nn.softmax))

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
Model = model.fit(X_train,
                         Y_train,
                         epochs=20,
                         batch_size=512,
                         validation_split = 0.2,
                         verbose = 2)

Epoch 1/20
875/875 - 361s - loss: 0.4762 - accuracy: 0.7719 - val_loss: 0.4361 - val_accuracy: 0.7951 - 361s/epoch - 413ms/step
Epoch 2/20
875/875 - 340s - loss: 0.4400 - accuracy: 0.7953 - val_loss: 0.4238 - val_accuracy: 0.8028 - 340s/epoch - 389ms/step
Epoch 3/20
875/875 - 338s - loss: 0.4300 - accuracy: 0.8012 - val_loss: 0.4182 - val_accuracy: 0.8065 - 338s/epoch - 386ms/step
Epoch 4/20
875/875 - 351s - loss: 0.4242 - accuracy: 0.8043 - val_loss: 0.4153 - val_accuracy: 0.8082 - 351s/epoch - 401ms/step
Epoch 5/20
875/875 - 337s - loss: 0.4202 - accuracy: 0.8065 - val_loss: 0.4133 - val_accuracy: 0.8092 - 337s/epoch - 386ms/step
Epoch 6/20
875/875 - 340s - loss: 0.4168 - accuracy: 0.8084 - val_loss: 0.4133 - val_accuracy: 0.8098 - 340s/epoch - 389ms/step
Epoch 7/20
875/875 - 339s - loss: 0.4141 - accuracy: 0.8102 - val_loss: 0.4114 - val_accuracy: 0.8100 - 339s/epoch - 387ms/step
Epoch 8/20
875/875 - 344s - loss: 0.4117 - accuracy: 0.8110 - val_loss: 0.4127 - val_accuracy: 0.8104 - 

In [ ]:
Model.model.save("senti.keras")

In [ ]:
Model.model.evaluate(np.array(X_test), np.array(Y_test))

15000/15000 [==============================] - 565s 38ms/step - loss: 0.4049 - accuracy: 0.8156


[0.40489232540130615, 0.8156145811080933]

In [ ]:
texts = ["This movie is great!",
         "I didn't like this film at all. It is bad",
         "The acting was okay, but the plot was boring."]

# Preprocess the input data
max_words = 10000  # or whatever the maximum number of words in your vocabulary is
maxlen = 118  # or whatever the maximum sequence length is

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to make them of the same length
data = pad_sequences(sequences, maxlen=maxlen)

In [ ]:
predictions = Model.model.predict(data)
# Interpret predictions
sentiment_labels = ['Negative', 'Positive']
for i, prediction in enumerate(predictions):
    predicted_label = sentiment_labels[np.argmax(prediction)]
    print(f"Text: {texts[i]}")
    print(f"Predicted Sentiment: {predicted_label} (Confidence: {prediction.max():.2f})")
    print()

1/1 [==============================] - 0s 44ms/step
Text: This movie is great!
Predicted Sentiment: Positive (Confidence: 0.56)

Text: I didn't like this film at all. It is bad
Predicted Sentiment: Negative (Confidence: 0.50)

Text: The acting was okay, but the plot was boring.
Predicted Sentiment: Negative (Confidence: 0.56)



#Testing

In [ ]:
import keras

senti = keras.models.load_model("senti.keras")

In [ ]:
texts = [
    "I love spending time with my family.",
    "The weather ruined our picnic.",
    "Eating ice cream always cheers me up." ,
]

labels = ["Positive", "Negative", "Positive",]

max_words = 10000
maxlen = 118

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences to make them of the same length
data = pad_sequences(sequences, maxlen=maxlen)

# Assuming you have a model named 'senti'
predictions = senti.predict(data)

# Interpret predictions
sentiment_labels = ['Negative', 'Positive']
for i, prediction in enumerate(predictions):
    predicted_label = sentiment_labels[np.argmax(prediction)]
    print(f"Text: {texts[i]}")
    print(f"Original Sentiment: {labels[i]}")
    print(f"Predicted Sentiment: {predicted_label} (Confidence: {prediction.max():.2f})")
    print()

1/1 [==============================] - 0s 45ms/step
Text: I love spending time with my family.
Original Sentiment: Positive
Predicted Sentiment: Positive (Confidence: 0.67)

Text: The weather ruined our picnic.
Original Sentiment: Negative
Predicted Sentiment: Positive (Confidence: 0.52)

Text: Eating ice cream always cheers me up.
Original Sentiment: Positive
Predicted Sentiment: Negative (Confidence: 0.69)

